In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('drive/My Drive/model_train/04 人脸识别')

In [0]:
!pip install numpy==1.16.2
!pip install pillow
!pip install scipy==1.1.0
!pip install facenet
!pip install align


In [0]:
import sys
sys.path.append('drive/My Drive/model_train/04 人脸识别/src/align')# 解决no module 'align.detect_face'
## 同时修改文件align_dataset_mtcnn.py的import align.detect_face ==> import detect_face ##
sys.path.append('drive/My Drive/model_train/04 人脸识别/src')
sys.path

## 修改src/align_dataset_mtcnn.py的import facenet ==> import facenet.src.facenet ## # 解决AttributeError: module 'facenet' has no attribute 'store_revision_info'
## 添加align_dataset_mtcnn.py的import align ## # NameError: name 'align' is not defined
## 修改align_dataset_mtcnn.py的58/100 line的align.detect_face ==> detect_face ## # AttributeError: module 'align' has no attribute 'detect_face'

## 修改contributed/predict.py的import align.detect_face ==> import align; import detect_face；还有内容里面对应的 ##
## 修改predict.py的import facenet ==> import facenet.src.facenet ## # 解决AttributeError: module 'facenet' has no attribute 'prewhiten'
## 然后把predict.py移到src路径下 ## 

#### 训练自己的模型------人脸检测与人脸对齐
主要是进行人脸对齐和人脸剪裁

In [25]:
!python src/align/align_dataset_mtcnn.py src/data/people src/data/people_160 --image_size 160 --margin 32 --random_order --gpu_memory_fraction 0.7

fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
Not a git repository
To compare two paths outside a working tree:
usage: git diff [--no-index] <path> <path>
Creating networks and loading parameters



2019-10-20 01:32:02.496308: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-10-20 01:32:02.496599: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2c34bc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-20 01:32:02.496637: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2019-10-20 01:32:02.498976: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-10-20 01:32:02.575694: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had

#### 训练自己的模型------模型学习1

In [27]:
!python src/classifier.py TRAIN src/data/people_160 src/models/20180402-114759 src/models/classifier.pkl


src/classifier.py:59: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(cls.image_paths)>0, 'There must be at least one image for each class in the dataset')

2019-10-20 01:52:13.532471: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-10-20 01:52:13.551671: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-10-20 01:52:13.552496: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2019-10-20 01:52:13.552757: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2019-10-20 01:52:13.554183: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Succes

#### 训练自己的模型------模型学习2
主要是进行模型分类结果的准确率测试

In [28]:
!python src/classifier.py CLASSIFY src/data/people_160 src/models/20180402-114759 src/models/classifier.pkl

src/classifier.py:59: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(cls.image_paths)>0, 'There must be at least one image for each class in the dataset')

2019-10-20 01:59:25.496221: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-10-20 01:59:25.515915: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-10-20 01:59:25.516709: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2019-10-20 01:59:25.516976: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2019-10-20 01:59:25.518487: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Succes

#### 训练自己的模型------效果验证
用单张图片进行测试

In [44]:
# !python src/predict.py src/data/test/liuyifei_01.jpg src/models/20180402-114759 src/models/classifier.pkl --gpu_memory_fraction 0.7
!python src/align/predict.py src/data/test/zhengshuang_01.jpg src/models/20180402-114759 src/models/classifier.pkl --gpu_memory_fraction 0.7


Creating networks and loading parameters



2019-10-20 02:46:03.882418: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-10-20 02:46:03.882670: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x30f0bc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-20 02:46:03.882706: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2019-10-20 02:46:03.885088: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-10-20 02:46:03.962426: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-10-20 02:46:03.963278: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x30f0d80 initialized for platform CUDA (this does not guarantee that XLA

In [45]:
!python src/classifier.py CLASSIFY src/data/test src/models/20180402-114759 src/models/classifier.pkl

src/classifier.py:59: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(cls.image_paths)>0, 'There must be at least one image for each class in the dataset')

2019-10-20 02:47:27.600135: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-10-20 02:47:27.619465: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-10-20 02:47:27.620209: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2019-10-20 02:47:27.620513: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2019-10-20 02:47:27.621837: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Succes